In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data)

In [ ]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    market_data_scaled,
    columns=['price_change_percentage_24h',
             'price_change_percentage_7d',	
             'price_change_percentage_14d',	
             'price_change_percentage_30d',
             'price_change_percentage_60d',
             'price_change_percentage_200d',
             'price_change_percentage_1y']
)

In [ ]:
# Copy the crypto names from the original data
df_market_data_scaled['coin_id'] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index('coin_id')

# Display sample data
df_market_data_scaled.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values
inertia1 = []

# Create a for loop to compute the inertia with each possible value of k
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=42)
    k_model.fit(df_market_data_scaled)
    inertia1.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data1 = {"k": k, "inertia": inertia1}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow1 = pd.DataFrame(elbow_data1)

#Review the DataFrame
df_elbow1.head()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow1.hvplot.line(
    x="k",
    y="inertia",
    title= "Elbow Curve 1",
    xticks=k
)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 3 or 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=3, random_state=42)

In [ ]:
# Fit the K-Means model using the scaled data
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(df_market_data_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
market_clusters = model.predict(df_market_data_scaled)

#Fit the model
model.fit(df_market_data_scaled)

# Print the resulting array of cluster values.
print(market_clusters)

In [ ]:
# Create a copy of the DataFrame
df_market_scaled_predictions = df_market_data_scaled.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_market_scaled_predictions["MarketCluster"]= market_clusters

# Display sample data
df_market_scaled_predictions.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
df_market_scaled_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="MarketCluster", # Color the graph points with the labels found using K-Means and 
    hover_cols= "coin_id" # identify the cryptocurrency represented by each data point.
)

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_pca = pca.fit_transform(df_market_scaled_predictions)

# View the first five rows of the DataFrame. 
market_pca[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

In [ ]:
0.34871677+0.31363391+0.22627118

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** About 89% of the total variance is explained by the three principal components.

In [ ]:
# Create a new DataFrame with the PCA data.
df_market_pca = pd.DataFrame(market_pca, columns=["PC1", "PC2", "PC3"])

# Copy the crypto names from the original data
df_market_pca['coin_id'] = df_market_data.index

# Set the coinid column as index
df_market_pca = df_market_pca.set_index('coin_id')

# Display sample data
df_market_pca.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values
inertia2 = []

# Create a for loop to compute the inertia with each possible value of k
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=42)
    k_model.fit(df_market_pca)
    inertia2.append(k_model.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data2 = {"k": k, "inertia": inertia2}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow2 = pd.DataFrame(elbow_data2)

#Review the DataFrame
df_elbow2.head()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow2.hvplot.line(
    x="k",
    y="inertia",
    title= "Elbow Curve 2",
    xticks=k
)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=42)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(df_market_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
market_clusters = model.predict(df_market_pca)

# Compute centers for predicted clusters
predicted_cluster_centers = pca_kmeans.cluster_centers_

# Print the resulting array of cluster values.
print(market_clusters)

In [ ]:
# Create a copy of the DataFrame with the PCA data
pca_predictions_df = df_market_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_predictions_df["PredictedClusters"] = market_clusters

# Display sample data
pca_predictions_df

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_predictions_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by= "PredictedClusters",
    hover_cols= ["coin_id"] 
)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
import holoviews as hv
hv.extension('bokeh')

# Composite plot to contrast the Elbow curves
elbow_curve1 = df_elbow1.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve Comparison",
    label="Elbow Curve 1",
    color='blue',
    xticks=k
)

# Create the second elbow curve plot
elbow_curve2 = df_elbow2.hvplot.line(
    x="k",
    y="inertia",
    label="Elbow Curve 2",
    color='red',
    xticks=k
)

# Combine the two plots into one composite plot
composite_plot = elbow_curve1 * elbow_curve2

# Display the composite plot
composite_plot.opts(title = "Elbow Curve Comparison")

In [ ]:
# Define distinct color palettes for each plot
color_palette1 = hv.Cycle(['teal', 'blue', 'yellow'])
color_palette2 = hv.Cycle(['maroon', 'orange', 'violet'])

# Composite plot to contrast the clusters
scatterplot1 = df_market_scaled_predictions.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="MarketCluster",
    hover_cols= "coin_id",
    color=color_palette1
)

scatterplot2 = pca_predictions_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by= "PredictedClusters",
    hover_cols= ["coin_id"],
    color= color_palette2
)

composite_scatter = scatterplot1 * scatterplot2

#Display the composite scatterplot
composite_scatter.opts(title= "Scatter Plot Comparison")

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 